In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
def read_and_clean_dataframe(file_path):
    df = pd.read_parquet(file_path)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    count_1 = df.duration.count()
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    count_2 = df.duration.count()
    print(f"Dropped: {count_1 - count_2}")
    
    df['PUlocationID'].fillna(-1, inplace=True)
    df['DOlocationID'].fillna(-1, inplace=True)

    na_percent = df[df['PUlocationID']==-1]["PUlocationID"].count()*100/df['PUlocationID'].count()

    print(f"N/A percentage: {na_percent:.2f}%")
    return df

In [3]:
df_train = read_and_clean_dataframe('data/fhv_tripdata_2021-01.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
df_valid = read_and_clean_dataframe('data/fhv_tripdata_2021-02.parquet')

In [ ]:
categorical = ['PUlocationID', 'DOlocationID']
df_train[categorical] = df_train[categorical].astype(str)
train_dicts = df_train[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

df_valid[categorical] = df_valid[categorical].astype(str)
valid_dicts = df_valid[categorical].to_dict(orient='records')

X_valid = dv.transform(valid_dicts)


In [ ]:
X_train.shape

In [ ]:
target = 'duration'
y_train = df_train[target].values

y_valid = df_valid[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_train)
y_pred_valid = lr.predict(X_valid)

In [ ]:
def plot(y_train, y_pred):
    sns.distplot(y_pred, label="prediction")
    sns.distplot(y_train, label="actual")
    plt.legend()

In [ ]:
# print("Training graph")
# plot(y_train, y_pred)
print("Evaluation graph")
plot(y_valid, y_pred_valid)

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_train, y_pred, squared=False)
print(f"Root mean squared error for training: {rmse}")
rmse = mean_squared_error(y_valid, y_pred_valid, squared=False)
print(f"Root mean squared error for validation: {rmse}")